[Home](../index.ipynb) / Web-Server
***
<span style="font-size:20pt;">Web-Server</span>

***
# ESP32 / ESP8266 als Accesspoint

**Achtung:** ESP8266: Ein **eigenes** Passwort (statt dem default Passwort **micropythoN**) braucht **mindestens 8 Zeichen**, andernfalls bricht das Programm ab mit der  
Fehlermeldung: _OSError: can't set AP config_.

Nach dem Start der Programme das WLAN des Controllers auswählen und die auf der Console angezeigte IP-Adresse ansurfen (meist 192.168.4.1).

## Beispiel: LED an/aus schalten

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 

import info
       
#######################################
# Constants

if   info.TYPE == "Esp8266 Croduino Nova" : ID_LED = 13 # Croduino
elif info.TYPE == "Esp32 HelTec"          : ID_LED = 25 # HeltTec
else : ID_LED = 14 # External LED

SSID        = "controller_a.fuchs"
PASSWORD    = '1231231123' # default is 'micropythoN', attension: at least 8 chars, otherwise error: 'OSError: can't set AP config'

MAX_CLIENTS = 5 # Good maximal ammount of posiible clients


STR_HTML_TOP = r"""<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="utf-8">
    <title>Web Server</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">
    <style>
        html{font-family:Helvetica;display:inline-block;margin:0px auto; text-align: center;}
        p{font-size: 2rem;}
        a{display:inline-block;border:none;border-radius:20px;color:white;padding:16px 20px;margin:2px;text-decoration:none}
        .clsOff{background-color:#eebb33;}
        .clsOn{background-color:#8080ee;}
    </style>
</head>
<body>
  <h1>Web Server</h1>
  <p>Switch LED:</p>"""
  

STR_HTML_BOTTOM = r"""</body>
</html>"""

#######################################
# Variables and init

from machine import Pin

led = Pin( ID_LED, Pin.OUT )


#======================================
# Typical code for setting up the controller as access point.
# At the end we get a socket listening for clients to connect:

import network
import socket


wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

if info.TYPE == "Esp8266 Croduino Nova" :
    wlanAccessPoint.config( essid = SSID, password = PASSWORD ) # Esp8266
else :
    wlanAccessPoint.config( essid = SSID, password = PASSWORD, max_clients = MAX_CLIENTS ) # ESP32
    
wlanAccessPoint.active( True )                  # activate the interface

print( wlanAccessPoint.ifconfig()[0] )

objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

try:
    objSocket.bind( ( '', 80 ) )
    objSocket.listen( MAX_CLIENTS )
except Exception as _e:
    print( _e )


#######################################
# Loop
# Listen for connecting clients and
# handle requests

try:
    while True:
        try :
            socketFrom, addressFrom = objSocket.accept() # The return value is a pair (socketFrom, addressFrom) where socketFrom is a new socket object usable to send 

            strRequest = str( socketFrom.recv( 1024 ) )

            iRequestPathStart = strRequest.find( "GET " ) # Every http request starts with "GET "

            if iRequestPathStart >= 0 :                    # followed by the request-path delimited with a ' ' charakter.
                iRequestPathStart += 4

                iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

                if iRequestPathEnd >= 0 :
                    strPath = strRequest[ iRequestPathStart : iRequestPathEnd ]
                    bIsOn   = False
                    # Now we have a valid request-path:
                    # print( "GET", strPath )
                    
                    if strPath == "/led/on" :
                        strPath = "/led/off"
                        led.on()
                        bIsOn = True
                    else :
                        strPath = "/led/on"
                        led.off()
                    
                    # print( "SEND", strPath )
                    
                    # This header is allways needed:
                    socketFrom.send   ( "HTTP/1.1 200 OK\n" )
                    socketFrom.send   ( "Content-Type: text/html; charset=UTF-8\n" )
                    socketFrom.send   ( "Connection: close\n\n" )
                    
                    # Now send html content:
                    socketFrom.sendall( STR_HTML_TOP   )
                    socketFrom.sendall( "<p><a class='cls{1}' href='{0}'>{1}</a></p>".format( strPath, "Off" if bIsOn else "On" ) )
                    socketFrom.sendall( STR_HTML_BOTTOM )

        except Exception as _e:
            print( _e )

        finally :
            try :
                socketFrom.close()
                # print( "Socket closed" )
            except: pass
        
except KeyboardInterrupt:
    try   : objSocket.close()
    except: pass

    try   : wlanAccessPoint.active( False )  
    except: pass


## Beispiel: Text auf das Display übertragen

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 

    
#######################################
# Constants

SSID        = "controller_a.fuchs"
PASSWORD    = '1231231123' # default is 'micropythoN', attension: at least 8 chars, otherwise error: 'OSError: can't set AP config'

MAX_CLIENTS = 5 # Good maximal ammount of posiible clients


STR_HTML_TOP = r"""<!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="utf-8">
    <title>Web Server</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">
    <style>
        html{font-family:Helvetica;display:inline-block;margin:0px auto;text-align:center;}
        input{display:inline-block;border-radius:20px;padding:12px 12px;margin:2px;font-size:1.2rem;}
    </style>
</head>
<body>
    <h1>Web Server</h1>
    <form action="/text">
        <input type="text" id="idText" name="idText" value="
"""

STR_HTML_BOTTOM = r""""><br><br>
        <input type="submit" value="Display text">
    </form>  
</body>
</html>"""


#######################################
# Variables and init

import display

display = display.Display()

display.clear()


# source: <a href="https://forum.micropython.org/memberlist.php?mode=viewprofile&u=552&sid=825359ab512e29152debfb452a578c20">Spotlight kid</a>
def uudecode( str ):
    bits = str.replace( "+", " " ).split('%')
    if len(bits) == 1:
        return str
    res = bytearray(bits[0])
    append = res.append
    extend = res.extend

    for item in bits[1:]:
        try:
            append(int(item[:2], 16))
            extend(item[2:])
        except KeyError:
            append(b'%')
            extend(item)

    return res.decode('utf-8')


#======================================
# Typical code for setting up the controller as access point.
# At the end we get a socket listening for clients to connect:

import network
import socket
import info


wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

if info.TYPE == "Esp8266 Croduino Nova" :
    wlanAccessPoint.config( essid = SSID, password = PASSWORD ) # Esp8266
else :
    wlanAccessPoint.config( essid = SSID, password = PASSWORD, max_clients = MAX_CLIENTS ) # ESP32
    
wlanAccessPoint.active( True )                  # activate the interface

print( wlanAccessPoint.ifconfig()[0] )
display.text( str(wlanAccessPoint.ifconfig()[0]), 0, 0 )
display.show()

objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

try:
    objSocket.bind( ( '', 80 ) )
    objSocket.listen( MAX_CLIENTS )
except Exception as _e:
    print( _e )


#######################################
# Loop
# Listen for connecting clients and
# handle requests

try:
    while True:
        try :
            socketFrom, addressFrom = objSocket.accept() # The return value is a pair (socketFrom, addressFrom) where socketFrom is a new socket object usable to send 

            strRequest = str( socketFrom.recv( 1024 ) )

            iRequestPathStart = strRequest.find( "GET " ) # Every http request starts with "GET "

            if iRequestPathStart >= 0 :                    # followed by the request-path delimited with a ' ' charakter.
                iRequestPathStart += 4

                iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

                if iRequestPathEnd >= 0 :
                    strText = strRequest[ iRequestPathStart + len("/text?idText="): iRequestPathEnd ]
                    #print( strText )
                    strText = uudecode( strText )
                    print( strText )
                    
                    display.scroll(0,-12)
                    display.fill_rect(0,52,127,63,0)
                    display.text( strText.replace("ä","ae").replace("ö","oe").replace("ü","ue").replace("Ä","Ae").replace("Ö","Oe").replace("Ü","Ue").replace("ß","ss"), 0, 52 )
                    display.show()
                    
                    # This header is allways needed:
                    socketFrom.send   ( "HTTP/1.1 200 OK\n" )
                    socketFrom.send   ( "Content-Type: text/html; charset=UTF-8\n" )
                    socketFrom.send   ( "Connection: close\n\n" )
                    
                    # Now send html content:
                    socketFrom.sendall( STR_HTML_TOP )
                    socketFrom.send   ( strText )
                    socketFrom.sendall( STR_HTML_BOTTOM )

        except Exception as _e:
            print( _e )

        finally :
            try   : socketFrom.close()
            except: pass
        
except KeyboardInterrupt:
    try   : objSocket.close()
    except: pass

    try   : wlanAccessPoint.active( False )  
    except: pass


---
## Einfache Klasse: SimpleAccessPointHttpServer

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 

In [ ]:
#================================================

MAX_REQUEST_LENGTH = 1024

#================================================

import network
import socket
import info
import ubinascii
import gc

#================================================
  
class SimpleAccessPointHttpServer:
    def __init__( self ) :
        self.objSocket  = None
        self.ipAddress  = "?.?.?.?"
        self.subnetMask = self.ipAddress
        self.gateway    = self.ipAddress
        self.dnsServer  = self.ipAddress
        self.macAddress = "?:?:?:?:?:?"

    def __str__( self ) :
        return "Name: " + self.__class__.__name__ + ", IP address: " + self.ipAddress + ", Subnet mask: " + self.subnetMask + ", Gateway: " + self.gateway + ", DNS server: " + self.dnsServer + ", MAC address: " + self.macAddress
        
    def start( self, ssid = "ESP32_Ajax_Sample", password = "1231231123", maxClients = 5 ):
        self.wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

        if info.TYPE == "Esp8266 Croduino Nova" :
            self.wlanAccessPoint.config( essid = ssid, password = password ) # Esp8266
        else :
            self.wlanAccessPoint.config( essid = ssid, password = password, max_clients = maxClients ) # ESP32

        self.wlanAccessPoint.active( True )                  # activate the interface

        print( self.wlanAccessPoint.ifconfig()[0] )

        self.objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

        try:
            self.objSocket.bind( ( '', 80 ) )
            self.objSocket.listen( maxClients )
            
            self.ipAddress, self.subnetMask, self.gateway, self.dnsServer = self.wlanAccessPoint.ifconfig() # [IP address, subnet mask, gateway, DNS server]
            self.macAddress = ubinascii.hexlify( network.WLAN().config('mac'),':' ).decode()
            
        except Exception as e:
            #print( "Exception of type {0} occurred. Arguments:\n{1!r}".format(type(e).__name__, e.args))
            print( e )

        gc.collect()

        
    def stop( self ):
        try : self.objSocket.close()
        except: pass
    
        try : self.wlanAccessPoint.active( False ) 
        except: pass
    
        gc.collect()

        
        
    def accept( self ):
        try :
            # socket.accept(): The return value is a pair (socketFrom, addressFrom)
            # where socketFrom is a new socket object usable to send 

            self.socketFrom = self.objSocket.accept()[ 0 ]
            
            if not self.dispatch(
                    self.getRequestPath( 
                        str( self.socketFrom.recv( MAX_REQUEST_LENGTH ) )
                    )
                ) :
                self.socketFrom.send( "HTTP/1.1 404 Not Found\n" )
                self.socketFrom.send( "Connection: close\n\n"    )

            
        except Exception as _e:
            print( _e )
            #print( "Exception of type {0} occurred. Arguments:\n{1!r}".format(type(_e).__name__, _e.args))
        
        finally :
            try : self.socketFrom.close()
            except: pass

    def sendHttpHeader( self, strContentType ) :
        self.send( "HTTP/1.1 200 OK\n"     )
        self.send( "Content-type: text/"   )
        self.send( strContentType          )
        self.send( "; charset=UTF-8\n"     )
        self.send( "Connection: close\n\n" )

    def sendHtmlHttpHeader( self ) :
        self.sendHttpHeader( "html" )
        
    def sendAjaxHttpHeader( self ) :
        self.sendHttpHeader( "plain" )
                    
        
    def getRequestPath( self, strRequest ) :
        iRequestPathStart = strRequest.find( "GET " )

        if iRequestPathStart >= 0 :
            iRequestPathStart += 4
            
            iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

            if iRequestPathEnd >= 0 :
                return strRequest[ iRequestPathStart : iRequestPathEnd ]
            
        return "/"
    
    
    def dispatch( self, strPath ) :
        return True
        
   
    def sendHtmlHead( self, strTitle = "AP", strJavaScript="", strCss = "", strAdditionalTags = "" ) :
        self.send( '<!DOCTYPE html><html lang="de"><head><meta charset="utf-8"><title>' )
        self.send( strTitle )
        self.send( '</title><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">' )
        self.send( '<script type="text/javascript">' )
        self.send( strJavaScript )
        self.send( '</script>' )
        self.send( '<style>' )
        self.send( strCss )
        self.send( '</style>' )
        self.send(  strAdditionalTags  )
        self.send( '</head>' )

    def send( self, strText ) :
        self.socketFrom.write( strText )


### Beispiel 1

In [ ]:
#######################################
# You have to owerwrite dispatch()

class HttpServer( SimpleAccessPointHttpServer ):
    def __init__( self ) :
        super().__init__()

    def dispatch( self, strPath ) :
        print( "dispatch: ", strPath )
        self.sendHtmlHttpHeader() 
        self.sendHtmlHead( strTitle = "Web Server", strJavaScript = "alert(Date());", strCss = "html{font-family:Helvetica;text-align:center;}" )
        self.send( "<body><h1>It works!</h1></body>" )
        self.send( "</html>" )
        return True

    
httpServer = HttpServer()
httpServer.start( ssid = "ESP32_Html_Sample", password = "1231231123" )
print( httpServer )


#######################################
# Loop


try :
    while True:
        print( "Waiting for client...", end="" )
        httpServer.accept()
        
except KeyboardInterrupt:
    httpServer.stop()
    print( "Server stopped." )

### Beispiel 2: Ajax

In [ ]:
import re
# CSS: Only for nice scrollbar:

STR_CSS = r'''
html{font-family:Helvetica;display:inline-block;margin:0px auto; text-align: center;}
.button{background-color:#4CAF50;border:none;color:white;padding:16px 40px;text-decoration:none;font-size:30px;margin:2px;cursor:pointer;}
.off{background-color:#555555;}
.slider { width:90%;}
input[type=range] { height: 57px; -webkit-appearance: none; margin: 10px 0; width: 100%;}
input[type=range]:focus { outline: none; }
input[type=range]::-webkit-slider-runnable-track { width: 100%; height: 15px; cursor: pointer; animate: 0.2s; box-shadow: 1px 1px 1px #000000; background: #3071A9; border-radius: 5px; border: 1px solid #000000;}
input[type=range]::-webkit-slider-thumb {box-shadow: 1px 1px 4px #000000;border: 0px solid #000000;height: 50px;width: 42px;border-radius: 7px;background: #FFC94A;cursor: pointer;-webkit-appearance: none;margin-top: -18px;}
input[type=range]:focus::-webkit-slider-runnable-track {background: #3071A9;}
input[type=range]::-moz-range-track {width: 100%;height: 15px;cursor: pointer;animate: 0.2s;box-shadow: 1px 1px 1px #000000;background: #3071A9;border-radius: 5px;border: 1px solid #000000;}
input[type=range]::-moz-range-thumb {box-shadow: 1px 1px 4px #000000;border: 0px solid #000000;height: 50px;width: 42px;border-radius: 7px;background: #FFC94A;cursor: pointer;}
input[type=range]::-ms-track {width: 100%;height: 15px;cursor: pointer;animate: 0.2s;background: transparent;border-color: transparent;color: transparent;}
input[type=range]::-ms-fill-lower {background: #3071A9;border: 1px solid #000000;border-radius: 10px;box-shadow: 1px 1px 1px #000000;}
input[type=range]::-ms-fill-upper {background: #3071A9;border: 1px solid #000000;border-radius: 10px;box-shadow: 1px 1px 1px #000000;}
input[type=range]::-ms-thumb {margin-top: 1px;box-shadow: 1px 1px 4px #000000;border: 0px solid #000000;height: 50px;width: 42px;border-radius: 7px;background: #FFC94A;cursor: pointer;}
input[type=range]:focus::-ms-fill-lower {background: #3071A9;}
input[type=range]:focus::-ms-fill-upper {background: #3071A9;}
'''


# JavaScript: Sends Ajax-requests.

STR_JAVASCRIPT = r'''
document.addEventListener(
    'DOMContentLoaded',
    function() { new RemoteControllApp(); },
    false
);

class RemoteControllApp
{
    constructor()
    {
        let htmlSliderSpeed          = document.getElementById( "idSliderSpeed" );
        let htmlLabelSliderSpeed     = document.getElementById( "idSliderSpeedLabel" );

        let htmlButtonStop           = document.getElementById( "idButtonStop" );

        let iLastValueSpeed     = htmlSliderSpeed.value;
        let bIsWaitingSpeed     = false; // only to avoid "overload"

        htmlLabelSliderSpeed.innerHTML     = htmlSliderSpeed.value;


        htmlButtonStop.addEventListener(
            "click",
            function() {
                htmlSliderSpeed.value = 0;
                htmlSliderSpeed.dispatchEvent(new Event('input', { bubbles: true }));
            }
        )

        htmlSliderSpeed.addEventListener(
            "input",
            function() {
                htmlSliderSpeed     .value     = this.value;
                htmlLabelSliderSpeed.innerHTML = this.value;
                let hTimeOut = null
                // Ajax:
                let xhttp = new XMLHttpRequest();
                xhttp.onreadystatechange = function() {
                    if (this.readyState == 4 && this.status == 200) {
                        bIsWaitingSpeed = false;
                        try {clearTimeout( hTimeOut )} catch (_e) {}
                        document.getElementById("idOutput").innerHTML = this.responseText;
                    }
                };

                if ( ! bIsWaitingSpeed )
                {
                    if ( iLastValueSpeed != this.value )
                    {
                        iLastValueSpeed = this.value
                        bIsWaitingSpeed = true;
                        hTimeOut = setTimeout(function(){bIsWaitingSpeed=false;},2000)
                        xhttp.open("GET", "/speed/" + this.value, true);
                        xhttp.send();
                    }
                }
            }
        )
    }
}
'''


STR_BODY = r'''
<body>
    <p>Speed: <span id="idSliderSpeedLabel"></span></p>
    <p><input type="range" min="-500" max="500" class="slider" id="idSliderSpeed"/></p>
    <p><input type="button" class="button" id="idButtonStop" value="Stop"/></p>
    <p id="idOutput"></p>
</body>
'''

PATTERN_SPEED     = re.compile( "/speed/(.*)" )

#######################################
# You have to owerwrite dispatch()

class HttpServer( SimpleAccessPointHttpServer ):
    def __init__( self ) :
        super().__init__()

    def dispatch( self, strPath ) :
        print( "dispatch: ", strPath )

        match = PATTERN_SPEED.match( strPath )
        if match :
            iSpeed = int(match.group( 1 ))
            
            self.sendAjaxHttpHeader()
            self.send( match.group( 1 ) )
            print( "Set speed: ", iSpeed )
            return True
        

        # All other requests are maped to "/"
        self.sendHtmlHttpHeader() 
        self.sendHtmlHead( strTitle = "Remote Control", strJavaScript = STR_JAVASCRIPT, strCss = STR_CSS )
        self.send( STR_BODY )
        self.send( "</html>" )
            
        return True

    
httpServer = HttpServer()
httpServer.start( ssid = "ESP32_Ajax_Sample", password = "1231231123" )
print( httpServer )


#######################################
# Loop


try :
    while True:
        print( "Waiting for client...", end="" )
        httpServer.accept()
        
except KeyboardInterrupt:
    httpServer.stop()
    print( "Server stopped." )